# Outline

- train word embeddings by using `matichon.json`
- use gensim `Word2Vec` model
- find similar words, calculate cosine similarity

# Import 

In [3]:
import pandas as pd
import numpy as np
import re

from gensim.models import Word2Vec

from pythainlp.tokenize import word_tokenize

- custom tokenization function
- remove all quatations and shrink newlines `\n` and white spaces 

In [5]:
def my_tokenize(text):
    text = re.sub(r'[\"\']', '', text) # remove quotations
    text = re.sub(r'[\n\t\s]', ' ', text) # shrink newlines/spaces
    tokens = word_tokenize(text, keep_whitespace=True)
    return tokens

# Load data

- need only article, drop the other columns
- tokenize article and store as list of tokens

In [12]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,เปิดใจชัดๆ กับ'ชูวิทย์' สบู่ ขัน ต้องเตรียมไว้...,นายชูวิทย์ กมลวิศิษฎ์ หัวหน้าพรรครักประเทศไทย ...,2015-08-16 10:58:27,politics,https://www.matichon.co.th/politics/news_104,104
1,ลุ้นบาททะลุ40ต่อดอลล์ ปลายปี'59 เหตุสงครามค่าเ...,แบงก์ซีไอเอ็มบี ชี้การฟื้นฟูเศรษฐกิจติดปัญหารา...,2015-08-26 11:19:03,economy,https://www.matichon.co.th/economy/news_148,148
2,"ศาลสูงแอฟริกาใต้ตัดสินให้ ""พิสตอเรียสผิดฐานฆาต...",จากรายงานของบีบีซี ออสการ์ พิสตอเรียส นักวิ่งร...,2015-12-04 02:52:25,foreign,https://www.matichon.co.th/foreign/news_247,247
3,แม่แดนผู้ดีฮึด! อุ้มท้องลูกแฝดแม้รู้ไม่รอด หวั...,เอ็มมา ลี คุณแม่ชาวอังกฤษ วัย 32 ปี ตัดสินใจอุ...,2015-12-04 03:11:37,foreign,https://www.matichon.co.th/foreign/news_268,268
4,"""ผบ.สส.""นำทหารรักษาพระองค์ สวนสนามเทิดพระเกียร...",เมื่อวันที่ 3 ธ.ค. ที่มณฑลพิธีท้องสนามหลวง พล....,2015-12-04 03:38:05,politics,https://www.matichon.co.th/politics/news_295,295
...,...,...,...,...,...,...
16321,'ชาคริต-เรย์' เพื่อนซี้พ่อลูกอ่อนรวมก๊วน งานนี...,เป็นเพื่อนที่สนิทกันมาแต่ไหนแต่ไร แถมพอมีลูก ก...,2018-07-05 11:43:53,entertainment,https://www.matichon.co.th/entertainment/news_...,1029324
16322,สงขลาจัดงานสมโภช 176 ปี ศาลเจ้าพ่อหลักเมือง,วันที่ 5 ก.ค. นายสมศักดิ์ ตันติเศรณี นายก ทน.ส...,2018-07-05 10:59:53,region,https://www.matichon.co.th/region/news_1029328,1029328
16323,'สมคิด' ลั่น ไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหาม...,"""สมคิด"" ลั่นไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหามิ...",2018-07-05 12:09:08,politics,https://www.matichon.co.th/politics/news_1029410,1029410
16324,ม.รามฯปฐมนิเทศ นศ.ใหม่ ตั้งใจศึกษาให้สำเร็จ-มี...,ผู้ช่วยศาสตราจารย์วุฒิศักดิ์ ลาภเจริญทรัพย์ อ...,2018-07-05 13:45:34,publicize,https://www.matichon.co.th/publicize/news_1029659,1029659


In [13]:
## drop column except for article
df = df.drop(columns=['date', 'headline', 'url','id'])

## tokenize
df['article_tokens'] = df['article'].apply(my_tokenize)

df

,article,category,article_tokens
0,นายชูวิทย์ กมลวิศิษฎ์ หัวหน้าพรรครักประเทศไทย ...,politics,"[นาย, ชู, วิทย์, , กมล, วิ, ศิษฎ์, , หัวหน้า..."
1,แบงก์ซีไอเอ็มบี ชี้การฟื้นฟูเศรษฐกิจติดปัญหารา...,economy,"[แบงก์, ซี, ไอ, เอ็ม, บี, , ชี้, การฟื้นฟู, เ..."
2,จากรายงานของบีบีซี ออสการ์ พิสตอเรียส นักวิ่งร...,foreign,"[จาก, รายงาน, ของ, บีบีซี, , ออสการ์, , พิ, ..."
3,เอ็มมา ลี คุณแม่ชาวอังกฤษ วัย 32 ปี ตัดสินใจอุ...,foreign,"[เอ็ม, มา, , ลี, , คุณแม่, ชาว, อังกฤษ, , ว..."
4,เมื่อวันที่ 3 ธ.ค. ที่มณฑลพิธีท้องสนามหลวง พล....,politics,"[เมื่อ, วันที่, , 3, , ธ.ค., , ที่, มณฑล, พ..."
...,...,...,...
16321,เป็นเพื่อนที่สนิทกันมาแต่ไหนแต่ไร แถมพอมีลูก ก...,entertainment,"[เป็นเพื่อน, ที่, สนิท, กัน, มา, แต่ไหนแต่ไร, ..."
16322,วันที่ 5 ก.ค. นายสมศักดิ์ ตันติเศรณี นายก ทน.ส...,region,"[วันที่, , 5, , ก.ค., , นาย, สม, ศักดิ์, ,..."
16323,"""สมคิด"" ลั่นไม่ใช่แค่สามมิตร แต่มีมวลหมู่มหามิ...",politics,"[สม, คิด, , ลั่น, ไม่, ใช่, แค่, สาม, มิตร, ..."
16324,ผู้ช่วยศาสตราจารย์วุฒิศักดิ์ ลาภเจริญทรัพย์ อ...,publicize,"[ผู้ช่วยศาสตราจารย์, วุฒิ, ศักดิ์, , ลาภ, เจ..."


# Train model

how to use :
[https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html)

- input of `Word2Vec` must be **list of list of tokens**
- `vector_size` : Dimensionality of the word vectors (usually 100-300)
- `window` : Maximum distance between the current and predicted word within a sentence
- `min_count` : Ignores all words with total frequency lower than this
- `sg` : Training algorithm: 1 for skip-gram; otherwise CBOW. (skip-gram is recommended)
- `epoch` : Number of iterations (epochs) over the corpus

In [43]:
## train
model = Word2Vec(sentences=df['article_tokens'], vector_size=100, window=5, min_count=3, sg=1, epochs=30)

# Word Embeddings

- `model.wv.most_similar(word, topn=xx)` gives the top N words with high cosine similarity 


In [45]:
model.wv.most_similar('อร่อย', topn=15)

[('รสชาติ', 0.8352956175804138),
 ('เมนู', 0.7987961769104004),
 ('ถูกปาก', 0.7343130707740784),
 ('ลิ้มลอง', 0.7331318855285645),
 ('หมูสับ', 0.7211061120033264),
 ('แซลมอน', 0.7199339270591736),
 ('บาร์บีคิว', 0.7173497080802917),
 ('สุกี้ยากี้', 0.7113039493560791),
 ('รสเปรี้ยว', 0.7110312581062317),
 ('กลมกล่อม', 0.7064458727836609),
 ('เครื่องเคียง', 0.7012359499931335),
 ('หวาน', 0.6998612880706787),
 ('ไข่เค็ม', 0.6992987394332886),
 ('ต้นตำรับ', 0.6863528490066528),
 ('เผ็ด', 0.6858212947845459)]

- `model.wv.similarity(word1, word2)` calculates the cosine similarity of the 2 words

In [46]:
model.wv.similarity('ผู้ชาย', 'ผู้หญิง')

0.8152483

- `model.wv` is like a dictionary. `model.wv[word]` gives the vector of the word

In [47]:
model.wv['อร่อย']

array([ 0.4113333 ,  0.11535873,  0.3334668 ,  0.634968  , -0.79331905,
        0.15287226, -0.3688044 ,  0.99255854, -0.08796303, -0.18494396,
       -0.67945313, -0.31490067,  0.12990588,  0.20420446, -0.10080083,
       -0.22775249,  0.39435497,  0.46160975,  0.6028755 ,  0.15873283,
        0.24331482, -0.09094347,  0.20604244, -0.24414434,  0.513986  ,
       -0.18136539, -0.29593354, -0.70221883, -0.50965405, -0.35629502,
        0.22213638,  0.22186008,  0.8405408 , -0.57750523, -0.01938633,
        0.48217067, -0.27887267,  0.07273053, -0.6433986 , -0.8796424 ,
       -0.1725576 , -0.07223132,  0.21523806,  0.17130417, -0.0285441 ,
        0.24849537, -0.46890974,  0.2727813 ,  0.04804863,  0.24120227,
       -0.14758715,  0.10566989, -0.50972575, -0.48751384, -0.35548687,
        0.5925638 , -0.27028757, -0.04879678, -0.85844374,  0.2868777 ,
        0.5999778 , -0.21262947,  0.22327693,  0.22273839, -0.02717081,
        0.17981869, -0.18467578, -0.2084532 ,  0.24859849,  0.69

- `model.wv.most_similar()` can add/subtract vectors
- use argument `positive` and `negative`, e.g. `model.wv.most_similar(positive=[w1, w2], negative=[w3])`

In [48]:
## ปักกิ่ง - จีน + เกาหลี
model.wv.most_similar(positive=['ปักกิ่ง', 'เกาหลี'], negative=['จีน'])

[('โซล', 0.5920867919921875),
 ('กรุง', 0.5373151898384094),
 ('ประชุมสุดยอด', 0.5288278460502625),
 ('อาบูดาบี', 0.5138489007949829),
 ('มุมไบ', 0.5113369226455688),
 ('เปียงยาง', 0.5070136785507202),
 ('ดูไบ', 0.5028671622276306),
 ('บริติช', 0.4936085641384125),
 ('ริยาด', 0.48732423782348633),
 ('มอสโคว์', 0.48513826727867126)]

# save & load model

In [44]:
## save the model
model.save("word2vec_matichon.model")

In [50]:
## load model 
model_loaded = Word2Vec.load("word2vec_matichon.model")
model_loaded.wv.most_similar('สวย')

[('เซ็กซี่', 0.6897033452987671),
 ('เข้าคู่', 0.6745166778564453),
 ('ฉายแวว', 0.6581156253814697),
 ('ชุดว่ายน้ำ', 0.65362948179245),
 ('ดูดี', 0.64291912317276),
 ('เย้ายวน', 0.6369034051895142),
 ('ความโหด', 0.6348311901092529),
 ('ชวนมอง', 0.6313931345939636),
 ('หวานฉ่ำ', 0.6252356767654419),
 ('เนี้ยบ', 0.6158441305160522)]